In [1]:
import json
json_string=""" {"ruleName":"Spacecraft Command Status Red","isSchedule":true,"Schedule":"1 Minute","scheduleTime":"1","scheduleTimePeriod":"Minute","filters":[{"topic_id":31,"trigger_operator":">","trigger_value":0.0,"filter":[{"name":"silverTimestamp","operator":"last","time":"1 Minute","filter_key_join_type":"AND"},{"name":"systemStatus","value":"RED","operator":"contains","filter_key_join_type":"AND"}],"group_key_join_type":"AND"}]}"""
data=json.loads(json_string)

In [2]:

import re
from datetime import datetime

from datetime import datetime, timedelta
def data_f():
    topic_id=data['filters'][0]['topic_id']
    first=data['filters'][0]
    first1=data['filters'][0]['filter']
    
    if 'trigger_operator' in first:
         trig_oper=data['filters'][0]['trigger_operator']
         trig_value=data['filters'][0]['trigger_value']
         filters_len=len(data['filters'][0]['filter'])

         string_list1=[]
         string_list2=[]
         string_list3=[]
         string_list4=[]
         
 
         start=datetime.now()
         print(start)        
   
         for  pos in range(0,filters_len):
                 
              if 'time' in  data['filters'][0]['filter'][pos]:
                      
                         name=data['filters'][0]['filter'][pos]['name']
                         join_key=data['filters'][0]['filter'][pos]['filter_key_join_type']
                         time=data['filters'][0]['filter'][pos]['time']
                         str = re.findall(r'\d+', time)
                         time_value_str=""
                         time_value_str+="".join(str)
                         time_value=int(time_value_str)
                         operator=data['filters'][0]['filter'][pos]['operator']
                   
                         # if operator is'last'
                         if operator=='last':
                              if 'Minute' in time:
                                   end = start - timedelta(minutes=time_value)
                              elif 'Second' in time:
                                   end = start - timedelta(seconds=time_value)
                              else:
                                   end =start - timedelta(hours=time_value)
                              string=f"({name} between '{end}' AND '{start}')" 
                         # else operator is others'             
                         else:
                              if 'Minute' in time:
                                   end = start + timedelta(minutes=time_value)
                                
                              elif 'Second' in time:
                                   end = start + timedelta(seconds=time_value)
                              else:
                                   end =start +timedelta(hours=time_value)
                              string=f"({name} between '{start}' AND '{end}')"

                         #if filter is at end of the position
                         if filters_len-1==pos:
                            string_t=string
                         # filter is at other position    
                         else:
                             string_t=f"{string}{join_key}"
                         # append the string to the list 
                         # string_list1.append(string_t)
                         # store in variable with if  the loop   // 
                         string=string_t    
                                  
                             
                    
              else:     
                         name=data['filters'][0]['filter'][pos]['name']
                         value=data['filters'][0]['filter'][pos]['value']
                         operator=data['filters'][0]['filter'][pos]['operator']
                         join_key=data['filters'][0]['filter'][pos]['filter_key_join_type']
                         # if operator is'last'
                         if operator == 'contains':
                              operator=f'like %{value}%'
                         elif operator =='startsWith' :
                               operator=f'like {value}%'
                         elif  operator =='endsWith' :
                               operator=f'like %{value}'

                         # else operator is others'         
                         else:
                               operator=f"{operator} {value}"


                         if filters_len-1==pos:
                           string=f"({name} {operator})"           
                            
                         else:
                            string=f"({name} {operator}){join_key}"
                         #append the string to the list      
                         # string_list1.append(string)
                         ###// store it with else of loop //   
                         # string=string   
                          
          #     string_list1.append(string)  # ERROR 2 times print  

              # combine the list                     
          #     string_list3=string_list1 +string_list2
          #     print(string_list3)
              string_list1.append(string) 
              result=""
              result+=" ".join(string_list1)          
                
         result_final= f"select case when (select count(*) from {topic_id} where {result} ){trig_oper} {trig_value} then 'True' else 'False' END AS EMAalert"     
         print(result_final)
                 
    else:
        aggreg_value=data['filters'][0]['aggregator_value']
        aggreg_oper=data['filters'][0]['aggregator_operator']
    
        name=data['filters'][0]['filter'][0]['name']
        aggregtr=data['filters'][0]['filter'][0]['aggregator']
        agg_name=data['filters'][0]['filter'][0]['aggregator_name']
        time=data['filters'][0]['filter'][0]['time']
        operator=data['filters'][0]['filter'][0]['operator']
        start=datetime.now()
        print(start)
        str = re.findall(r'\d+', time)
        time_value_str=""
        time_value_str+="".join(str)
        time_value=int(time_value_str)
        if operator =='last':
               # print("true")
               if 'Minute' in time:
                                   end = start - timedelta(minutes=time_value)
                                   
               elif 'Second' in time:
                                   end = start - timedelta(seconds=time_value)
               else:
                                   end =start - timedelta(hours=time_value)
               string_t=f"{name} between '{end}' AND '{start}'"         
        else:
               if 'Minute' in time:
                                   end = start + timedelta(minutes=time_value)
                                   
               elif 'Second' in time:
                                   end = start + timedelta(seconds=time_value)
               else:
                                   end =start + timedelta(hours=time_value)
               string_t=f"{name} between '{start}' AND '{end}'"  
        
        result1=string_t       

        result=f"select case when(select {aggregtr}({agg_name}) from {topic_id} where ( {result1})){aggreg_oper}{aggreg_value} then 'True' else 'False' END AS EMAAggregrate_Alert"
        print(result)
    
                    
    
           
           

In [3]:
data_f()

2024-01-08 11:07:29.225138
select case when (select count(*) from 31 where (silverTimestamp between '2024-01-08 11:06:29.225138' AND '2024-01-08 11:07:29.225138')AND (systemStatus like %RED%) )> 0.0 then 'True' else 'False' END AS EMAalert
